## Финальное соревнование

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

В данном задании вас ждет неизвестная зависимость. Ваша основная задача: **построить две лучших модели**, минимизирующих среднеквадратичную ошибку (MSE):
1. На первую модель не налагается ограничений.
2. Вторая модель должна быть **линейной**, т.е. представлять собой линейную комбинацию признаков плюс свободный член: $\boldsymbol{w}^{\top}\boldsymbol{x} + b$. При этом __вы можете использовать базовые математические операции для преобразования признаков__: np.exp, np.log, np.pow (полный список доступен в [документации](https://numpy.org/doc/stable/reference/routines.math.html)), а также линейные операции над ними (сумма, умножение на число и пр.). Для преобразования признаков вам будет необходимо написать функцию `my_transformation`. __Кол-во параметров (весов) используемых второй моделью не должно превышать 15 (включая свободный член).__

Настоятельно рекомендуем написать код "с нуля", лишь поглядывая на готовые примеры, а не просто "скопировать-вставить". Это поможет вам в будущем писать код более уверенно

In [1]:
import os
import json

import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Загрузка данных происходит ниже. Если она не срабатывает, самостоятельно скачайте файл `hw_final_open_data.npy` и положите его в ту же директорию, что и ноутбук.

In [ ]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy -O hw_final_open_data.npy
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy -O hw_final_open_data.npy

In [70]:
assert os.path.exists('hw_final_open_data.npy'), 'Please, download `hw_final_open_data.npy` and place it in the working directory'
assert os.path.exists('hw_final_open_target.npy'), 'Please, download `hw_final_open_target.npy` and place it in the working directory'
data = np.load('hw_final_open_data.npy', allow_pickle=False)
target = np.load('hw_final_open_target.npy', allow_pickle=False)

Разбивка на `train` и `val` опциональна и сделана для вашего удобства.

In [3]:
data[0,:]

array([-0.56581437, -0.43054397, -0.56863048, -0.37948571, -0.5274203 ,
       -0.16069121, -0.57543757])

In [4]:
data = preprocessing.normalize(data)

In [6]:
data[0,:]

array([-0.44620333, -0.33952858, -0.44842412, -0.29926385, -0.41592562,
       -0.12672169, -0.45379222])

In [ ]:
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = feature_matrix[:, 0] * feature_matrix[:, 1]
    return new_feature_matrix

In [71]:
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25,random_state=42)

### Модель №1
Напоминаем, в первой части задания ваша основная задача – получить наилучший результат без ограничений на модель. Сдаваться будут только предсказания модели.

Пример с использованием Random Forest доступен ниже.

In [8]:
from catboost import CatBoostRegressor, EShapCalcType, EFeaturesSelectionAlgorithm
from catboost import Pool

In [9]:
train_pool = Pool(train_x, train_y)
test_pool = Pool(valid_x, valid_y)

In [53]:

model = CatBoostRegressor(iterations=1000, random_seed=42, learning_rate=0.1)
summary = model.select_features(
        train_pool,
        eval_set=test_pool,
        features_for_select=list(range(train_x.shape[1])),     
        num_features_to_select=3,  
        steps=50,                                   
        algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
        shap_calc_type=EShapCalcType.Regular,           
        train_final_model=True,                        
        plot=False
    )
print(summary)


The number of features selection steps (50) is greater than the number of features to eliminate (4). The number of steps was reduced to 4.


Step #1 out of 4
0:	learn: 1.9619025	test: 2.1021712	best: 2.1021712 (0)	total: 1.96ms	remaining: 1.96s
1:	learn: 1.7835594	test: 1.9144791	best: 1.9144791 (1)	total: 3.79ms	remaining: 1.89s
2:	learn: 1.6281132	test: 1.7468587	best: 1.7468587 (2)	total: 5.75ms	remaining: 1.91s
3:	learn: 1.4812129	test: 1.5906898	best: 1.5906898 (3)	total: 8.21ms	remaining: 2.04s
4:	learn: 1.3488373	test: 1.4504786	best: 1.4504786 (4)	total: 10ms	remaining: 2s
5:	learn: 1.2331353	test: 1.3260320	best: 1.3260320 (5)	total: 11.8ms	remaining: 1.96s
6:	learn: 1.1280445	test: 1.2135469	best: 1.2135469 (6)	total: 13.8ms	remaining: 1.95s
7:	learn: 1.0347349	test: 1.1145732	best: 1.1145732 (7)	total: 16ms	remaining: 1.98s
8:	learn: 0.9483882	test: 1.0198114	best: 1.0198114 (8)	total: 18.1ms	remaining: 1.99s
9:	learn: 0.8704333	test: 0.9365429	best: 0.9365429 (9)	total: 20.3ms	remaining: 2.01s
10:	learn: 0.8003083	test: 0.8606691	best: 0.8606691 (10)	total: 22.4ms	remaining: 2.02s
11:	learn: 0.7374761	test: 0.79

In [11]:
rf = CatBoostRegressor( task_type="GPU",
                           devices='0:1',iterations=5000,max_depth=5,random_seed=300, learning_rate=0.1)
rf.fit(train_x, train_y,early_stopping_rounds=10)

print(
    f'train mse =\t {mean_squared_error(np.round(rf.predict(train_x), 2), np.round(train_y)):.5f}',
    f'validation mse = {mean_squared_error(np.round(rf.predict(valid_x)), np.round(valid_y)):.5f}',
    sep='\n'
)

0:	learn: 1.9789575	total: 8.23ms	remaining: 41.1s
1:	learn: 1.7984820	total: 19.5ms	remaining: 48.8s
2:	learn: 1.6355017	total: 32.4ms	remaining: 54s
3:	learn: 1.4889576	total: 40.8ms	remaining: 51s
4:	learn: 1.3560647	total: 50ms	remaining: 50s
5:	learn: 1.2378542	total: 58.7ms	remaining: 48.8s
6:	learn: 1.1291844	total: 66.9ms	remaining: 47.7s
7:	learn: 1.0325003	total: 77ms	remaining: 48s
8:	learn: 0.9474970	total: 84.7ms	remaining: 47s
9:	learn: 0.8712387	total: 93.8ms	remaining: 46.8s
10:	learn: 0.8026510	total: 106ms	remaining: 48.2s
11:	learn: 0.7412899	total: 119ms	remaining: 49.3s
12:	learn: 0.6842447	total: 128ms	remaining: 49.1s
13:	learn: 0.6332354	total: 137ms	remaining: 48.8s
14:	learn: 0.5900846	total: 148ms	remaining: 49.1s
15:	learn: 0.5499717	total: 160ms	remaining: 49.8s
16:	learn: 0.5140292	total: 172ms	remaining: 50.3s
17:	learn: 0.4833392	total: 182ms	remaining: 50.5s
18:	learn: 0.4561041	total: 190ms	remaining: 49.8s
19:	learn: 0.4328410	total: 202ms	remaining: 

In [72]:
import xgboost as xg

In [73]:
xgb_r = xg.XGBRegressor( 
                  n_estimators = 120, seed = 123)

xgb_r.fit(train_x, train_y) 
print(
    f'train mse =\t {mean_squared_error(np.round(xgb_r.predict(train_x), 2), np.round(train_y)):.5f}',
    f'validation mse = {mean_squared_error(np.round(xgb_r.predict(valid_x)), np.round(valid_y)):.5f}',
    sep='\n'
)

train mse =	 0.13590
validation mse = 0.32500


In [51]:
xgb_r.feature_importances_.sort()
from sklearn.feature_selection import SelectFromModel

In [74]:

for thresh in xgb_r.feature_importances_:
    selection = SelectFromModel(xgb_r, threshold=thresh, prefit=True)
    select_X_train = selection.transform(train_x)
    selection_model =  xg.XGBRegressor()
    selection_model.fit(select_X_train, train_y)
    select_X_test = selection.transform(valid_x)
    print(
        
        f'train mse =\t {mean_squared_error(np.round(selection_model.predict(select_X_train), 2), np.round(train_y)):.5f}',
        f'validation mse = {mean_squared_error(np.round(selection_model.predict(select_X_test)), np.round(valid_y)):.5f}',
        sep='\n'
    )


train mse =	 0.13640
validation mse = 0.32000
train mse =	 0.13640
validation mse = 0.32000
train mse =	 0.13640
validation mse = 0.32000
train mse =	 0.13640
validation mse = 0.32000
train mse =	 0.13640
validation mse = 0.32000
train mse =	 0.13640
validation mse = 0.32000
train mse =	 0.13640
validation mse = 0.32000


##### Сдача первой части соревнования
Загрузите файл `hw_final_closed_data.npy` (ссылка есть на странице с заданием). Если вы используете sklearn-совместимую модель, для генерации посылки вы можете воспользоваться функцией `get_predictions`. В ином случае перепишите функцию для вашей модели и запустите код под следующей ячейкой для генерации посылки.

In [ ]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_closed_data.npy -O hw_final_closed_data.npy

In [46]:
assert os.path.exists('hw_final_closed_data.npy'), 'Please, download `hw_final_closed_data.npy` and place it in the working directory'
closed_data = np.load('hw_final_closed_data.npy', allow_pickle=False)

Если необходимо, преобразуйте данные. Преобразованную матрицу объект-признак сохраните в переменную `closed_data`.

In [ ]:
# optional transformations

In [ ]:
def get_predictions(model, eval_data, step=10):
    predicted_values = model.predict(eval_data)
    return predicted_values

Обращаем ваше внимание, предсказания округляются до сотых!

In [ ]:
predicted_values = np.round(get_predictions(model=rf, eval_data=closed_data), 2)

assert predicted_values.shape == (closed_data.shape[0], ) # predictions should be just one-dimensional array

In [ ]:
# do not change the code in the block below
# __________start of block__________
def float_list_to_comma_separated_str(_list):
    _list = list(np.round(np.array(_list), 2))
    return ','.join([str(x) for x in _list])

submission_dict = {
    'predictions': float_list_to_comma_separated_str(predicted_values)
}
with open('submission_dict_final_p01.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
    
print('File saved to `submission_dict_final_p01.npy`')
# __________end of block__________

### Модель №2
Функция `my_transformation` принимает на вход матрицу объект-признак (`numpy.ndarray` типа `np.float`) и преобразует ее в новую матрицу. Данная функция может использовать только numpy-операции, а также арифметические действия.

Для примера доступна функция ниже. Она лишь добавляет новый признак, представляющий собой произведение первого и второго исходных признаков (считая с нуля).

In [ ]:
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+4))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = np.cos(feature_matrix[:,0]*feature_matrix[:,4]*feature_matrix[:,5])
    new_feature_matrix[:, -2] = np.sin(feature_matrix[:,0])
    new_feature_matrix[:, -3] = np.sin(feature_matrix[:,4])
    new_feature_matrix[:, -4] = np.sin(feature_matrix[:,5])
    print(new_feature_matrix.shape)
    return new_feature_matrix

In [ ]:
transformed_train_x = my_transformation(train_x)

In [ ]:
lr = Ridge()
lr.fit(transformed_train_x, train_y)

print(
    f'train mse =\t {mean_squared_error(lr.predict(transformed_train_x), train_y):.5f}',
    f'validation mse = {mean_squared_error(lr.predict(my_transformation(valid_x)), valid_y):.5f}',
    sep='\n'
)

Обращаем ваше внимание, что параметры линейной модели будут округляться до __четырех знаков после запятой__. Это не должно сильно повлиять на качество предсказаний:

In [ ]:
original_predictions = lr.predict(transformed_train_x)
rounded_predictions = transformed_train_x.dot(np.round(lr.coef_, 4)) + np.round(lr.intercept_, 4)


assert np.allclose(original_predictions, rounded_predictions, atol=1e-3)

Параметры вашей модели:

In [ ]:
w_list = list(np.round(lr.coef_, 4))
print(f'w = {list(np.round(lr.coef_, 4))}\nb = {np.round(lr.intercept_, 4)}')

Напоминаем, ваша модель не должна использовать более 15 параметров (14 весов плюс свободный член).

In [ ]:
assert len(w_list) + 1 <= 15

##### Сдача второй части соревнования
Для сдачи вам достаточно отправить функцию `my_transformation` и параметры вашей модели в контест в задачу №2. Пример посылки доступен ниже. Имортирование `numpy` также необходимо.

In [ ]:
# __________example_submission_start__________
import numpy as np
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = feature_matrix[:, 0
    ] * feature_matrix[:, 1]
    return new_feature_matrix

w_submission = [-0.0027, -0.2637, 0.0, -0.1134, -0.0165, -0.9329, 0.0, 0.1293]
b_submission = 1.1312
# __________example_submission_end__________

На этом задание завершено. Поздравляем!